
# Trading Strategy Simulator

This notebook demonstrates how to build a trading strategy simulator using Python.
It covers data fetching, signal generation for simple strategies, backtesting with transaction costs and slippage, performance evaluation, parameter search, and a multi-asset portfolio example.
The code attempts to fetch historical market data via the `yfinance` package and falls back to synthetic data if downloading fails, ensuring the notebook runs anywhere.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Try to import yfinance; if not available, we'll simulate data.
try:
    import yfinance as yf
    yfinance_available = True
except ImportError:
    yfinance_available = False


In [ ]:
def fetch_data(tickers, start="2018-01-01", end="2025-01-01"):
    """
    Fetch adjusted close price data for a list of tickers using yfinance.
    If yfinance is unavailable or returns no data, synthetic price series are generated via geometric Brownian motion.

    Parameters:
        tickers (list of str): List of ticker symbols.
        start (str): Start date in 'YYYY-MM-DD' format.
        end (str): End date in 'YYYY-MM-DD' format.
    Returns:
        pd.DataFrame: DataFrame of closing prices indexed by date with one column per ticker.
    """
    data = {}
    if yfinance_available:
        for ticker in tickers:
            try:
                df = yf.download(ticker, start=start, end=end)
                if df.empty:
                    raise ValueError("No data returned")
                # Use Adj Close if present
                if 'Adj Close' in df.columns:
                    df['Close'] = df['Adj Close']
                df = df[['Close']]
                df.columns = [ticker]
                data[ticker] = df
            except Exception as e:
                print(f"Failed to fetch {ticker}: {e}")
    # Fallback to synthetic data if nothing fetched
    if not data:
        print("Using synthetic data via geometric Brownian motion since data fetch failed.")
        dates = pd.date_range(start=start, end=end, freq='B')
        for ticker in tickers:
            n = len(dates)
            mu, sigma = 0.1, 0.2  # drift and volatility
            dt = 1/252
            prices = np.zeros(n)
            prices[0] = 100
            for i in range(1, n):
                prices[i] = prices[i-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.normal())
            data[ticker] = pd.DataFrame({ticker: prices}, index=dates)
    # Combine into a single DataFrame and forward fill missing values
    df_all = pd.concat(data.values(), axis=1).ffill().dropna()
    return df_all


In [ ]:
# Example: fetch Apple stock prices (or synthetic data) from 2018 to 2025
prices_df = fetch_data(["AAPL"], start="2018-01-01", end="2025-01-01")
prices_df.head()


In [ ]:
def sma_crossover_signals(prices, short_window=20, long_window=50):
    """
    Compute a simple moving average (SMA) crossover signal.
    When the short-term moving average is above the long-term moving average, the signal is +1 (long); otherwise it is -1 (short).

    Parameters:
        prices (pd.Series): Series of prices.
        short_window (int): Lookback period for the short-term SMA.
        long_window (int): Lookback period for the long-term SMA.
    Returns:
        pd.Series: Series of trading signals (+1 for long, -1 for short).
    """
    df = pd.DataFrame({'price': prices})
    df['sma_short'] = df['price'].rolling(window=short_window).mean()
    df['sma_long'] = df['price'].rolling(window=long_window).mean()
    signal = np.where(df['sma_short'] > df['sma_long'], 1, -1)
    return pd.Series(signal, index=prices.index)


In [ ]:
def compute_rsi(prices, period=14):
    """Compute the Relative Strength Index (RSI)."""
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def rsi_strategy(prices, rsi_period=14, lower=30, upper=70):
    """
    Generate a trading signal based on RSI.
    When RSI falls below `lower`, a long position is entered (+1).
    When RSI rises above `upper`, a short position is entered (-1).
    The previous signal is held until the next entry condition is met.
    """
    rsi = compute_rsi(prices, rsi_period)
    raw_signal = np.where(rsi < lower, 1, np.where(rsi > upper, -1, 0))
    current = 0
    signals = []
    for s in raw_signal:
        if s != 0:
            current = s
        signals.append(current)
    return pd.Series(signals, index=prices.index)


In [ ]:
def backtest(prices, signals, trade_cost=0.0, slippage=0.0):
    """
    Run a vectorized backtest of a trading strategy.

    Parameters:
        prices (pd.Series): Series of prices.
        signals (pd.Series): Series of positions (+1 for long, -1 for short) aligned to prices.
        trade_cost (float): Proportional transaction cost applied whenever the position changes.
        slippage (float): Additional proportional cost per trade representing slippage.
    Returns:
        pd.DataFrame: DataFrame with returns and cumulative equity curves.
    """
    df = pd.DataFrame(index=prices.index)
    df['price'] = prices
    df['returns'] = df['price'].pct_change().fillna(0)
    df['signal'] = signals.shift(1).fillna(0)
    df['strategy_returns'] = df['signal'] * df['returns']
    trades = df['signal'].diff().abs().fillna(0)
    df['strategy_returns'] -= trades * (trade_cost + slippage)
    df['cumulative_market'] = (1 + df['returns']).cumprod()
    df['cumulative_strategy'] = (1 + df['strategy_returns']).cumprod()
    return df


In [ ]:
def compute_metrics(equity_curve, periods_per_year=252):
    """
    Compute a suite of performance metrics from a backtest equity curve.

    Returns a dictionary containing total return, CAGR, Sharpe ratio, Sortino ratio, maximum drawdown, Calmar ratio, hit rate, average win/loss and exposure.
    """
    returns = equity_curve['strategy_returns']
    total_return = equity_curve['cumulative_strategy'].iloc[-1] - 1
    n_periods = len(returns)
    years = n_periods / periods_per_year
    cagr = equity_curve['cumulative_strategy'].iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan
    sharpe = np.sqrt(periods_per_year) * returns.mean() / returns.std() if returns.std() != 0 else np.nan
    downside = returns[returns < 0]
    sortino = np.sqrt(periods_per_year) * returns.mean() / downside.std() if downside.std() != 0 else np.nan
    cum = equity_curve['cumulative_strategy']
    running_max = cum.cummax()
    drawdown = cum / running_max - 1
    max_drawdown = drawdown.min()
    calmar = (total_return / -max_drawdown) if max_drawdown != 0 else np.nan
    positives = returns[returns > 0]
    negatives = returns[returns < 0]
    hit_rate = len(positives) / (len(positives) + len(negatives)) if (len(positives) + len(negatives)) > 0 else np.nan
    avg_win = positives.mean() if len(positives) > 0 else np.nan
    avg_loss = negatives.mean() if len(negatives) > 0 else np.nan
    exposure = equity_curve['signal'].abs().mean()
    return {
        'Total Return': total_return,
        'CAGR': cagr,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Max Drawdown': max_drawdown,
        'Calmar Ratio': calmar,
        'Hit Rate': hit_rate,
        'Avg Win': avg_win,
        'Avg Loss': avg_loss,
        'Exposure': exposure,
    }


In [ ]:
# Use the SMA crossover strategy on Apple data
apple_prices = prices_df['AAPL']
signals = sma_crossover_signals(apple_prices, short_window=20, long_window=50)
backtest_df = backtest(apple_prices, signals, trade_cost=0.001)
metrics = compute_metrics(backtest_df)

# Display metrics
metrics_df = pd.DataFrame([metrics])
metrics_df


In [ ]:
# Plot equity curves
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(backtest_df.index, backtest_df['cumulative_market'], label='Buy & Hold')
ax.plot(backtest_df.index, backtest_df['cumulative_strategy'], label='Strategy')
ax.set_title('Cumulative Returns: Market vs. SMA Strategy')
ax.set_ylabel('Cumulative Return')
ax.legend()
plt.show()

# Plot drawdown
fig, ax = plt.subplots(figsize=(10, 3))
drawdown = backtest_df['cumulative_strategy'] / backtest_df['cumulative_strategy'].cummax() - 1
ax.plot(backtest_df.index, drawdown)
ax.set_title('Drawdown')
ax.set_ylabel('Drawdown')
plt.show()


In [ ]:
# Grid search for SMA parameters on Apple data
short_windows = [10, 20, 30]
long_windows = [50, 100, 200]
results = []
for short_win in short_windows:
    for long_win in long_windows:
        if short_win < long_win:
            sig = sma_crossover_signals(apple_prices, short_window=short_win, long_window=long_win)
            bt_df = backtest(apple_prices, sig, trade_cost=0.001)
            met = compute_metrics(bt_df)
            met.update({'Short Window': short_win, 'Long Window': long_win})
            results.append(met)
results_df = pd.DataFrame(results).set_index(['Short Window', 'Long Window'])
# Display results sorted by Sharpe ratio
results_df.sort_values('Sharpe Ratio', ascending=False)


In [ ]:
# Multi-asset equal-weighted portfolio example
# Fetch multiple tickers
multi_prices = fetch_data(['AAPL', 'MSFT', 'GOOGL'], start='2018-01-01', end='2025-01-01')
# Normalize each price series to start at 1
normed = multi_prices / multi_prices.iloc[0]
# Equal weights
weights = np.ones(len(normed.columns)) / len(normed.columns)
# Construct portfolio price by summing weighted normalized prices
portfolio_price = (normed * weights).sum(axis=1)
# Generate signals on the portfolio price
portfolio_signals = sma_crossover_signals(portfolio_price, short_window=20, long_window=50)
# Backtest
portfolio_bt = backtest(portfolio_price, portfolio_signals, trade_cost=0.001)
# Compute metrics
portfolio_metrics = compute_metrics(portfolio_bt)

# Display results
portfolio_metrics_df = pd.DataFrame([portfolio_metrics])
portfolio_metrics_df

# Plot portfolio equity curve
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(portfolio_bt.index, portfolio_bt['cumulative_market'], label='Buy & Hold (Portfolio)')
ax.plot(portfolio_bt.index, portfolio_bt['cumulative_strategy'], label='Strategy (Portfolio)')
ax.set_title('Cumulative Returns: Portfolio vs. SMA Strategy')
ax.set_ylabel('Cumulative Return')
ax.legend()
plt.show()



## Conclusion

In this notebook you built a simple but extensible trading strategy simulator.
You learned how to fetch or simulate price data, implement moving-average and RSI strategies, run vectorized backtests with transaction costs and slippage, compute risk-adjusted performance metrics and visualize results.
You also explored a basic parameter grid search and constructed an equal-weighted multi-asset portfolio.

From here you can extend the simulator by:

- Implementing other technical indicators or machine-learning based signals.
- Incorporating dynamic position sizing rules (e.g. volatility targeting or Kelly criterion).
- Adding out-of-sample testing and walk-forward analysis to avoid overfitting.
- Building a simple interactive dashboard using Streamlit or Dash for experiment control and visualization.

Always remember to validate your strategy on unseen data and be aware of the limitations of historical backtests.
